In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy
from numpy import delete
import os

In [2]:
path_test_kpi_raw = r"C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Confirmatory_Experiments\Instances\Test"
path_results = r"C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Confirmatory_Experiments\Results\PBMH_results\GA_rival_results"

In [3]:
# Generator/Selector of PIs
def instances(lib, instance_number, number_objects=100, difficulty="EASY"):
    fileName = path_test_kpi_raw + f"\\GA-{lib}_{difficulty}_{number_objects}_{instance_number:03d}.kp"
    with open(fileName, "r") as f:
        lines = f.readlines()
    line = lines[0].split(",")
    nbItems = int(line[0].strip())
    k_limit = int(line[1].strip())
    PI = [None] * nbItems
    for i in range(nbItems):
        line = lines[i + 1].split(",")
        weight = int(line[0].strip())
        profit = float(line[1].strip())
        PI[i] = (profit, weight)
    return PI, k_limit

In [4]:
def createIndividual(PI, k_limit):
    wgt = 0
    solution = [0] * len(PI)
    # Shuffle the indices to randomize the order in which items are considered
    indices = list(range(len(PI)))
    random.shuffle(indices)
    for index in indices:
        value, weight = PI[index]
        if weight <= k_limit and wgt + weight <= k_limit:
            decision = random.randint(0, 1)  # 0 or 1 for equal chance
            if decision:
                wgt += weight
                solution[index] = 1
    return solution

In [5]:
def combine(parentA, parentB, cRate, PI, k_limit):
    if random.random() > cRate:
        return parentA.copy(), parentB.copy()

    offspringA = [0] * len(parentA)
    offspringB = [0] * len(parentB)

    # Calculate the total weight of the knapsack for each gene being 1
    def calculate_weight(individual):
        return sum(weight for i, (value, weight) in enumerate(PI) if individual[i] == 1)

    # Iterate over genes
    for i in range(len(parentA)):
        # For offspringA
        if parentA[i] == 1 and calculate_weight(offspringA) + PI[i][1] <= k_limit:
            offspringA[i] = 1
        elif parentB[i] == 1 and calculate_weight(offspringA) + PI[i][1] <= k_limit:
            offspringA[i] = 1

        # For offspringB
        if parentB[i] == 1 and calculate_weight(offspringB) + PI[i][1] <= k_limit:
            offspringB[i] = 1
        elif parentA[i] == 1 and calculate_weight(offspringB) + PI[i][1] <= k_limit:
            offspringB[i] = 1

    return offspringA, offspringB

In [6]:
def mutate(individual, mRate, PI, k_limit):
    individual_copy = individual.copy()
    
    # Calculate the current weight of the individual
    def calculate_current_weight(ind):
        return sum(PI[i][1] for i in range(len(ind)) if ind[i] == 1)

    current_weight = calculate_current_weight(individual_copy)
    
    # Mutate the individual with consideration for the weight limit
    for i in range(len(individual)):
        if random.random() <= mRate:
            # If the item is currently not included and adding it doesn't exceed the weight limit
            if individual_copy[i] == 0 and current_weight + PI[i][1] <= k_limit:
                individual_copy[i] = 1  # Include the item
                current_weight += PI[i][1]  # Update the current weight
            elif individual_copy[i] == 1:  # If the item is currently included, remove it
                individual_copy[i] = 0
                current_weight -= PI[i][1]  # Update the current weight

    # The function returns the mutated individual, ensuring it doesn't exceed the weight limit
    return individual_copy

In [7]:
# Evaluator of Solutions
def evaluator(PI, solution, k_limit):
    s1 = (0, 0, 0)
    for i in range(len(PI)): # Iterates all the objects in the problem instance
        if solution[i] == 1: # When the object is in the knapsack considers the 
            # object in the evaluation
            s1 = (k_limit, s1[1] + PI[i][0], s1[2] + PI[i][1]) # Sums up the profit and 
            # the weight of all the items
    if s1[2] <= k_limit: # When the knapsack is not broken saves a record of 0
        s1 = (k_limit, s1[1], s1[2], 0, solution)
    else:
        s1 = (k_limit, s1[1], s1[2], 1, solution) # When the knapsack is broken saves a record of 1
    return s1

In [8]:
# Tournament selection # It was changed
def select(population, evaluation, tSize):
  winner = numpy.random.randint(0, len(population)) # Assigns as the winner a 
  # random number (index) between 0 and the length of the population
  for i in range(tSize - 1): # Iterates from 0 to the tournament size minus 1
    rival = numpy.random.randint(0, len(population)) # Assigns as the rival a
    # random number (index) between 0 and the length of the population
    if (evaluation[rival][1] > evaluation[winner][1]): # When the rival 
      # value/profit is larger than the winner value/profit executes:
      winner = rival # Assigns the rival index as the winner
  return population[winner] # Returns the winner of the population

In [9]:
# Evolutionary Process, Genetic Algorithm
def geneticAlgorithm(PI, k_limit, lib, PI_num, population_size, generations, cRate, mRate):
  population = []
  evaluations = []
  #Population Generator
  for i in range(population_size):
    population.append(createIndividual(PI, k_limit))
    evaluations.append(evaluator(PI, population[i], k_limit))
  population_size = len(population)
  # Keeps a record of the best individual found so far
  index = 0;
  for i in range(1, population_size):
    if (evaluations[i][1] > evaluations[index][1]):
      index = i;      
  bestIndividual = population[index]
  bestEvaluation = evaluations[index]
  
  # Keeps the information for plotting the performance of the algorithm
  best = [0] * generations
  avg = [0] * generations

  # Evolutionary Process
  for i in range(generations):
    newPopulation = []
    
    # Crossover
    for j in range(population_size // 2):
      parentA = select(population, evaluations, 3)
      parentB = select(population, evaluations, 3)
      offspring1, offspring2 = combine(parentA, parentB, cRate, PI, k_limit)
      newPopulation.append(offspring1)
      newPopulation.append(offspring2)
    population = newPopulation
    population_size = len(population)
    
    # Mutation
    for j in range(population_size):
      population[j] = mutate(population[j], mRate, PI, k_limit)
      evaluations[j] = evaluator(PI, population[j], k_limit)
    
    # Broken Solutions Killer
    population_size = len(population)
    
    # Keeps a record of the best individual found so far
    for j in range(population_size):
      if (evaluations[j][1] > bestEvaluation[1]):
        bestEvaluation = evaluations[j]
        bestIndividual = population[j]
      best[i] = bestEvaluation[1]
      avg[i] = numpy.average([evaluation[1] for evaluation in evaluations])
  plt.plot(range(generations), best, label = "Best")
  plt.plot(range(generations), avg, label = "Average")
  plt.legend()
  plt.xlabel('Generation Number')
  plt.ylabel('Profit Units')
  plt.savefig(path_results + f'\\GA_plots\GAKP_{lib}_{PI_num}_Results.png',
  transparent = False, dpi = 300)
  plt.close()
  return bestIndividual, bestEvaluation

In [10]:
def solution_assembler(number_objects=100, difficulty="EASY"):
    libs = ['DEF', 'MAXPW', 'MAXP', 'MINW']
    PI_nums = [str(num) for num in range(number_objects)] 
    PI_labels = [f'GA-{lib}_{difficulty}_100_{str(num).zfill(3)}' for lib in libs for num in range(number_objects)]
    bestSolutions = []
    bestEvaluations = []
    for lib in libs:   
        for i in range(len(PI_nums)):
            PI, k_limit = instances(lib, int(PI_nums[i]))
            solution, evaluation = geneticAlgorithm(PI, k_limit, lib, PI_nums[i], 
            population_size = len(PI), generations = 1000, cRate = 0.001, 
            mRate = 0.001)
            bestEvaluations.append(evaluation)
            bestSolutions.append(solution)
    df = pd.DataFrame(bestEvaluations, columns=["Knapsack Limit", "Profit", "Weight", "Knapsack State", "Solution"])
    df['Solution'] = bestSolutions
    df.index = PI_labels
    df.index.names = ['Problem Instance']
    df.to_csv(path_results + f'\\GAKP_Results.csv', encoding='utf-8')
    print("Data has been successfully written to GAKP_results.csv")
    return df

In [11]:
df = solution_assembler()

Data has been successfully written to GAKP_results.csv
